In [ ]:
from ophyd import PointGreyDetector, SingleTrigger
from ophyd import ADComponent, ImagePlugin, PixiradDetector, SingleTrigger
from ophyd.areadetector.filestore_mixins import FileStoreHDF5IterativeWrite, FileStoreTIFFIterativeWrite
from ophyd.areadetector.plugins import HDF5Plugin_V34, TIFFPlugin
import os

DET_FILES_ROOT = "S:\\mpe_feb24\\tomo_in_e\\" #NOTE: Windows machine has special directory 
BLUESKY_FILES_ROOT = "/home/beams/S1IDUSER/mnt/s1c/" #vantage point of bluesky reading the files
TEST_IMAGE_DIR = "mpe_feb24/tomo_in_e/" #directory where images go 

class MyHDF5Plugin(FileStoreHDF5IterativeWrite, HDF5Plugin_V34): ...
class MyTIFFPlugin(FileStoreTIFFIterativeWrite, TIFFPlugin): ...

class s1PointGrey(SingleTrigger, PointGreyDetector):
    """Try making a camera with a trigger."""
    # hdf1 = ADComponent(
    #     MyHDF5Plugin,
    #     "HDF1:",    
    #     write_path_template=os.path.join(DET_FILES_ROOT, TEST_IMAGE_DIR),
    #     read_path_template=os.path.join(BLUESKY_FILES_ROOT, TEST_IMAGE_DIR),
    # )
    
    #image = ADComponent(ImagePlugin, "IMAGE1:")    #TODO: check image plugin pv is correct
    
    tiff1 = ADComponent(
        MyTIFFPlugin,
        "TIFF1:",
        #write_path_template = os.path.join(DET_FILES_ROOT, TEST_IMAGE_DIR), #would work for linux machine
        write_path_template = DET_FILES_ROOT,
        read_path_template = os.path.join(BLUESKY_FILES_ROOT, TEST_IMAGE_DIR))

    #Simple tiff plugin
    #tiff1 = ADComponent(TIFFPlugin, "TIFF1:")
    
test_det = s1PointGrey("1idPG1:", name = "test_det")   


#Staging params here
#test_det.hdf1.create_directory.put(-5)
# test_det.cam.stage_sigs["acquire_time"] = 0.3
# test_det.cam.stage_sigs["acquire_period"] = 0.4
# test_det.cam.stage_sigs["num_images"] = 20
# test_det.cam.stage_sigs["image_mode"] = 'Multiple'


In [13]:
#Complex tiff1 plugin format
test_det.tiff1.stage_sigs

OrderedDict([('enable', 1),
             ('blocking_callbacks', 'Yes'),
             ('parent.cam.array_callbacks', 1),
             ('auto_increment', 'Yes'),
             ('array_counter', 0),
             ('auto_save', 'Yes'),
             ('num_capture', 0),
             ('file_template', '%s%s_%6.6d.tiff'),
             ('file_write_mode', 'Single')])

In [11]:
#Simple tiff1 plugin format
test_det.tiff1.stage_sigs

OrderedDict([('enable', 1),
             ('blocking_callbacks', 'Yes'),
             ('parent.cam.array_callbacks', 1)])

In [17]:
#staging, triggering, and unstaging

test_det.stage()
#st = test_det.trigger()
#test_det.unstage()

In [ ]:
st.wait()

In [15]:
test_det.unstage()

In [ ]:
#HDF plugin staging params (copied from pilatus example)

test_det.hdf1.stage_sigs["lazy_open"] = 1
#det.hdf1.stage_sigs["compression"] = "LZ4"
test_det.hdf1.stage_sigs["file_template"] = "%s%s_%3.3d.h5"
del test_det.hdf1.stage_sigs["capture"]  #LazyOpening setting must happen before plugin capture; change order in dictionary
test_det.hdf1.stage_sigs["capture"] = 1  #set as last action

In [ ]:
##Trying to acquire an save an image

#try this to see if lazyopen worked
test_pixi.stage()

#do this if you got an error
#det.warmup()

In [ ]:
#find the file that was written by the IOC
test_det.hdf1.full_file_name.get()